In [ ]:
import numpy as np
import pandas as pd
# from pandas import Series, DataFrame 
import seaborn as sns
from matplotlib import pyplot as plt

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

import warnings
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.model_selection import cross_validate, train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV, validation_curve

# from xgboost import XGBClassifier
import lightgbm as ltb
# from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import joblib
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier


warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
pd.set_option('display.width', 10000)


In [ ]:
df=pd.read_csv(r"data/maas.csv")

In [ ]:
df.head()

In [ ]:
#okunurluk için kolon isimlerini büyük harf yaptık.
df.columns = [col.upper() for col in df.columns]
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
#her kolonun içerdiği unique değer sayısına bakıyoruz.
for col in df.columns:
    print(col, df[col].nunique())

In [ ]:
#her kolondaki unique değerlerin neler olduğuna bakıyoruz.
for col in df.columns:
    print(col, df[col].unique())
    print(f"============================================================================================")


In [ ]:
#birbiri ile alakalı ve az sayılı pozisyon tiplerini birleştirdik.Daha anlamlı sonuşlar almamız için bu işlemi yapıyoruz.
#DATA SCIENTIST POZİSYONU İLE ALAKALI OLANLAR
ds= ["Data Scientist", "Machine Learning", "Data Analyst", "Data Engineer", "Computer Vision Engineer"]
for j in ds:
    df[df["POSITION"] == j]
    for i in df[df["POSITION"] == j].index:
        df["POSITION"][i] = "DATA SCIENTIST" 


In [ ]:
# DATABASE ADMİN  İLE ALAKALI OLANLAR
for i in df[df["POSITION"]=="SAP/ABAP/ERP Developer"].index:
    df["POSITION"][i] = "DATABASE ADMIN"

for i in df[df["POSITION"]=="Database Administrator"].index:
    df["POSITION"][i] = "DATABASE ADMIN"

for i in df[df["POSITION"]=="SysAdmin"].index:
    df["POSITION"][i] = "DATABASE ADMIN"

In [ ]:
#SAYILARI ÇOK AZ OLAN POZİSYONLAR
diger=["Project & Product Manager / PO", "Support Engineer", "Business Analyst", "RPA Developer", "IT Specialist", "R&D Engineer", "Security", "Consultant", "Seo Specialist", "Blockchain Developer"]
for j in diger:
    df[df["POSITION"] == j]
    for i in df[df["POSITION"] == j].index:
        df["POSITION"][i] = "OTHERS"

In [ ]:
df.head()

In [ ]:
#tüm pozisyon isimlendirmeleri standart hale gelsin diye büyük harf yaptık.
df["POSITION"] = [i.upper() for i in df["POSITION"]]

In [ ]:
df["POSITION"].value_counts()

In [ ]:
df["WORK_TYPE"].value_counts()

In [ ]:
#okunaklılık için pandemi nedeniyle ibareleri silindi.
for i in df[df["WORK_TYPE"]=="Pandemi Nedeniyle; Geçici Olarak Hibrit (Ofis + Remote)"].index:
    df["WORK_TYPE"][i] = "Geçici Hibrit"

for i in df[df["WORK_TYPE"]=="Pandemi Nedeniyle; Geçici Olarak Remote"].index:
    df["WORK_TYPE"][i] = "Geçici Remote"

for i in df[df["WORK_TYPE"]=="Hibrit (Ofis + Remote)"].index:
    df["WORK_TYPE"][i] = "Hibrit"

In [ ]:
df["WORK_TYPE"].value_counts()

In [ ]:
df["COMPANY"].value_counts()

In [ ]:
#SAYISI ÇOK AZ VEYA BENZER OLAN COMPANY KOLONLARINI BİRLEŞTİRDİK.
list_company= ["diğer", "kobi", "kamu", "Banka", "Telekomünikasyon","Danışmanlık","Savunma & Güvenlik","Kobi","Kamu","Diğer"]
for i in list_company:
    df[df["COMPANY"] == i]
    for j in df[df["COMPANY"] == i].index:
        df["COMPANY"][j] = "DIGER"

In [ ]:
df["COMPANY"].value_counts()

In [ ]:
#TL OLAN MAAŞLARI DÜZENLİYORUZ.
num=[]

for item in df["SALARY_FOR_TL_CURRENCY"]:
    if item:
        num.append(str(item).split(" "))

In [ ]:
#TL tipinden minimum maaş bilgilerini aldım.NaN ifadeler yerine 0 atadım.Değerleri doğru almak için 1000 ile çarptım
num1=[]
for i in range(0,len(num)):
    if num[i][0]=="nan":
        num1.append(0)
    else:
        num1.append(float(num[i][0])*1000)

In [ ]:
#TL tipinden maximum maaş bilgilerini aldım,
# NaN ifadeler yerine 0 atadım, doğru değerleri elde etmek için 1000 ile çarptım.
# Max ve Min değerleri aynı olmasın diye içerisinde üzeri yazan değerlerin üst sınırını 60000 tl aldım.
num2=[]
for i in range(0,len(num)):
    if num[i][0]=="nan":
        num2.append(0)
    elif (num[i][3])=='üzeri':
        num2.append(60000.0)
    else:
        num2.append(float(num[i][3])*1000)

In [ ]:
#veri setimize min tl ve max tl kazançlarını gösteren kolonları ekledik.
df["MIN_TL"]=num1

df["MAX_TL"]=num2

In [ ]:
#TL dışındaki para birimlerini parçaladım.
cur=[]

for item in df["SALARY_FOR_OTHER_CURRENCY"]:
    if item:
        cur.append(str(item).split(" "))

In [ ]:
#Döviz tipi min maaş bilgilerini çektim.1000 ile çarptım. yanlış girilen 1.0 YERİNE 1000 döviz birimini aynı bıraktım
cur1=[]
for i in range(0,len(cur)):
    if cur[i][0]=="nan":
        cur1.append(0)
    elif cur[i][0]=="1000":
        cur1.append(float(cur[i][0]))
    else:
        cur1.append(float(cur[i][0])*1000)

In [ ]:
#Döviz tipi max maaş bilgilerini çektim.1000 ile çarptım. Üst sınırı 10999 kabul ettim.(TL de 60000 almıştım.)
cur2=[]
for i in range(0,len(cur)):
    if cur[i][0]=="nan":
        cur2.append(0)
    elif (cur[i][2])=='üzeri':
        cur2.append(10999)#float(cur[i][0])
    else:
        cur2.append(float(cur[i][2])*1000)

In [ ]:
df["CUR_MIN"]=cur1
df["CUR_MAX"]=cur2

In [ ]:
df.head()

In [ ]:
#bu işlemlerle kazanılan miktarı sayısal bir değişken yapmış olduk.
df.info()

In [ ]:
#12 Aralık 2022 tarihli Sterlin Dolar ve Euro kurlarıyla Döviz cinsi TL ye çevrildi.
# >>>>Bu değerler 0 olan MIN_TL lere atandı.
#bu şekilde tüm maaş değerleri TL olarak kıyaslanabilecek noktasına geldik.
(df.loc[df["CURRENCY"]=="£ - Sterlin","MIN_TL"])=(df.loc[df["CURRENCY"]=="£ - Sterlin"]["CUR_MIN"]*22.56)
(df.loc[df["CURRENCY"]=="$ - Dolar","MIN_TL"])=(df.loc[df["CURRENCY"]=="$ - Dolar"]["CUR_MIN"]*18.63)
(df.loc[df["CURRENCY"]=="€ - Euro","MIN_TL"])=(df.loc[df["CURRENCY"]=="€ - Euro"]["CUR_MIN"]*19.47)

In [ ]:
#BENZER İŞLEMİ MAX_TL İÇİN DE YAPIYORUZ
(df.loc[df["CURRENCY"]=="£ - Sterlin","MAX_TL"])=(df.loc[df["CURRENCY"]=="£ - Sterlin"]["CUR_MAX"]*22.56)+(df.loc[df["CURRENCY"]=="£ - Sterlin"]["MAX_TL"])
(df.loc[df["CURRENCY"]=="$ - Dolar","MAX_TL"])=(df.loc[df["CURRENCY"]=="$ - Dolar"]["CUR_MAX"]*18.63)+(df.loc[df["CURRENCY"]=="$ - Dolar"]["MAX_TL"])
(df.loc[df["CURRENCY"]=="€ - Euro","MAX_TL"])=(df.loc[df["CURRENCY"]=="€ - Euro"]["CUR_MAX"]*19.47)+(df.loc[df["CURRENCY"]=="€ - Euro"]["MAX_TL"])

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.isnull().sum()

In [ ]:
df.loc[df["MIN_TL"]==df["MIN_TL"].min()].index

In [ ]:
#maaş bilgisi hiç girilmemiş kişileri sildik.
df=df.drop(df.loc[df["MIN_TL"]==df["MIN_TL"].min()].index)

In [ ]:
df.describe().T

In [ ]:
df.head()

In [ ]:
def grab_col_names(dataframe, cat_th=15, car_th=20):
    """

    Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.
    Not: Kategorik değişkenlerin içerisine numerik görünümlü kategorik değişkenler de dahildir.

    Parameters
    ------
        dataframe: dataframe
                Değişken isimleri alınmak istenilen dataframe
        cat_th: int, optional
                numerik fakat kategorik olan değişkenler için sınıf eşik değeri
        car_th: int, optinal
                kategorik fakat kardinal değişkenler için sınıf eşik değeri

    Returns
    ------
        cat_cols: list
                Kategorik değişken listesi
        num_cols: list
                Numerik değişken listesi
        cat_but_car: list
                Kategorik görünümlü kardinal değişken listesi

    Examples
    ------
        import seaborn as sns
        df = sns.load_dataset("iris")
        print(grab_col_names(df))


    Notes
    ------
        cat_cols + num_cols + cat_but_car = toplam değişken sayısı
        num_but_cat cat_cols'un içerisinde.
        Return olan 3 liste toplamı toplam değişken sayısına eşittir: cat_cols + num_cols + cat_but_car = değişken sayısı

    """

    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car, num_but_cat

In [ ]:
cat_cols, num_cols, cat_but_car, num_but_cat= grab_col_names(df)

In [ ]:
def cat_summary(dataframe, col_name, plot=True):
    print(pd.DataFrame({col_name: dataframe[col_name].value_counts(),
                        "Ratio": 100 * dataframe[col_name].value_counts() / len(dataframe)}))
    print("##########################################")
    if plot:
        sns.countplot(x=dataframe[col_name], data=dataframe) #gorsellestirmek istiyorsak yukarıdaki "plot=False"u True yaparız
        plt.show()

In [ ]:
for col in cat_cols:
    cat_summary(df, col)

In [ ]:
#GENDER daire grafigi yüzdelik
labels = ("Erkek", "Kadın")
sizes = [4497/5030, 533/5030]
plt.pie(sizes, labels=labels, autopct='%1.2f%%')
plt.title("Percent of GENDER")
plt.show()

In [ ]:
#POSİTION SAYILARININ DAĞILIMI

df["POSITION"].value_counts().plot(kind='bar')
plt.title("Count of POSITION")
plt.show(block=True)

In [ ]:
df["COMPANY"].value_counts().plot(kind="bar")
plt.title("WHICH COMPANY")
plt.show(block=True)

In [ ]:
#WORK_TYPE LEVEL KIRILIMINDA DEĞERLERİ
df.groupby(["WORK_TYPE", "LEVEL"]).agg({"LEVEL": "count"})

In [ ]:
#POZİSYONLARIN CİNSİYET KIRILIMINDA DEĞERLERİ
df.groupby(["POSITION", "GENDER"]).agg({"POSITION": "count"})

In [ ]:
#POZİSYONLARIN LEVEL KIRILIMINDA MIN MAX MAAŞLAR VE POZİSYONDA ÇALIŞAN SAYISI
df.groupby(["POSITION","LEVEL"]).agg({"MIN_TL": "mean", "MAX_TL": "mean","POSITION":"count"})

In [ ]:
#LEVEL IN WORK_TYPE KIRILIMINDA MIN MAX DEĞERLERİ VE SAYILARI
df.groupby(["LEVEL","WORK_TYPE"]).agg({"MIN_TL": "mean", "MAX_TL": "mean","WORK_TYPE":"count"})

In [ ]:
df.info()

In [ ]:
#TARGET degiskenimizi olusturuyoruz
df["SALARY_AVG_TL"] = (df["MIN_TL"] + df["MAX_TL"]) / 2
df["SALARY_AVG_TL"] = df["SALARY_AVG_TL"].astype("int")

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df["CITY"].unique()

In [ ]:
#Şehirleri yurt dışı ve yurt içi olarak güncelleyelim
ydisi=['Yurt Dışı', 'Sydney', 'Krakow', 'Miami', 'İngiltere', 'Düsseldorf ', 'Köln', 'Amsterdam', 'Belçika', 'Stuttgart ', 'Luxembourg', 'Utrecht', 'Isviçre', 'Seattle',
       'Nürnberg', 'San Fransisco', 'Londra', 'Stokholm', 'Tallinn', 'ABD', 'Berlin', 'Washington', 'amsterdam', 'Singapur', 'Kaliforniya', 'Ukrayna ', 'Paris', 'İtalya', 
       'Londra ', 'Letonya', 'Dubai', 'Berlin ', 'Indianapolis', 'costarica', 'Bratislava ', 'Zurich', 'Sofia', 'Dublin', 'Almanya', 'İsviçre', 'Kiev', 'Los Angeles', 'Kopenhag ', 'texas',
       'Palo Alto', 'Amsterdam ', 'New York', 'KKTC', 'MISIR', 'Montreuil', 'Helsinki', 'Varşova', 'Irvine', 'Brüksel', 'Kuzey Amerika', 'Oslo', 'Münih', 'Connecticut']

In [ ]:
for j in ydisi:
    df[df["CITY"] == j]
    for i in df[df["CITY"] == j].index:
        df["CITY"][i] = "yurt_disi"

In [ ]:
for i in df[df["CITY"]!="yurt_disi"].index:
    df["CITY"][i] = "yurt_ici"

In [ ]:
df["CITY"].unique()

In [ ]:
df.head()

In [ ]:
#kullanmayacagımız degiskenleri df'ten cıkarıyorum
silcols=["TECH_STACK", "COMPANY_SIZE", "SALARY_FOR_OTHER_CURRENCY", "SALARY_FOR_TL_CURRENCY", "MIN_TL", "MAX_TL", "CUR_MIN", "CUR_MAX"]

In [ ]:
df.drop(silcols, axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
cat_cols, num_cols, cat_but_car, num_but_cat= grab_col_names(df)

In [ ]:
cat_cols

In [ ]:
df[cat_cols].nunique()

In [ ]:
# Sınıflar arası fark varsa (ordinal kategorik değişkenler)- Label Encoding
#le_cols = ["LEVEL", "EXPERIENCE"]

# Sınıflar arası fark yoksa (nominal kategorik değişkenler) - One Hot Encoding
#ohe_cols = ["POSITION", "GENDER", "COMPANY", "WORK_TYPE", "CITY"]

In [ ]:
#kategorik degiskenlerde bicimlendirme yapıyoruz
def one_hot_encoder(dataframe, categorical_cols, drop_first=False):
    dataframe = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=drop_first)
    return dataframe
#drop_first=True yaparsak label encoder olarak kullanabiliriz,iki sınıflı kategorik degiskenleri de label encoder a sokabiliyor olacak

In [ ]:
df = one_hot_encoder(df, cat_cols, drop_first=True)

In [ ]:
df.head()

In [ ]:
#SÜTUN İSİMLERİ EKRAN GİRİŞ MODÜLÜNE GÖRE TEKRARDAN SIRALANDI
df = df[['SALARY_AVG_TL', 'LEVEL_Middle', 'LEVEL_Senior', 'POSITION_FULL STACK DEVELOPER','POSITION_FRONT-END DEVELOPER', 'POSITION_MOBILE APPLICATION DEVELOPER', 'POSITION_TEAM / TECH LEAD', 'POSITION_DATA SCIENTIST', 'POSITION_SOFTWARE DEVELOPMENT MANAGER', 'POSITION_SOFTWARE ARCHITECT', 'POSITION_EMBEDDED SOFTWARE DEVELOPER', 'POSITION_DEVOPS ENGINEER', 'POSITION_QA / TEST AUTOMATION DEVELOPER', 'POSITION_GAME DEVELOPER', 'POSITION_CTO', 'POSITION_DATABASE ADMIN', 'POSITION_OTHERS', 'EXPERIENCE_1 - 3 Yıl', 'EXPERIENCE_10 - 14 Yıl', 'EXPERIENCE_15 Yıl ve üzeri', 'EXPERIENCE_3 - 5 Yıl', 'EXPERIENCE_5 - 7 Yıl', 'EXPERIENCE_7 - 10 Yıl', 'GENDER_Kadın', 'COMPANY_Kurumsal', 'COMPANY_Startup', 'COMPANY_E-Ticaret', 'COMPANY_Outsource', 'COMPANY_Dijital / Reklam Ajansı', 'COMPANY_Oyun', 'COMPANY_Yazılım & Teknoloji', 'COMPANY_Fintech / Finans', 'WORK_TYPE_Remote', 'WORK_TYPE_Hibrit', 'WORK_TYPE_Ofis', 'WORK_TYPE_Geçici Remote', 'CITY_yurt_ici', 'CURRENCY_₺ - Türk Lirası', 'CURRENCY_€ - Euro', 'CURRENCY_£ - Sterlin']]
df.head()

In [ ]:
df.columns

In [ ]:
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### Tail #####################")
    print(dataframe.tail(head))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)


In [ ]:
#MODEL KURMA

In [ ]:
#Bagımlı ve bagımsız degiskenlerimizi ayırıyoruz
y = df["SALARY_AVG_TL"]
X = df.drop(["SALARY_AVG_TL"], axis=1)

In [ ]:
# Degiskenlerin önem düzeyini belirten feature importance
def plot_importance(model, features, num=len(X), save=False):

    feature_imp = pd.DataFrame({"Value": model.feature_importances_, "Feature": features.columns})
    plt.figure(figsize=(10, 10))
    sns.set(font_scale=1)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False)[0:num])
    plt.title("Features")
    plt.tight_layout()
    plt.show()
    if save:
        plt.savefig("importances.png")

model = LGBMRegressor()
model.fit(X, y)

plot_importance(model, X)

In [ ]:
## Train verisi ile model kurup, model başarısını değerlendirelim
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=17)

In [ ]:
#MODELRMSE DEGERLERİNE BAKALIM
models = [('LR', LinearRegression()),
          #("Ridge", Ridge()),
          #("Lasso", Lasso()),
          #("ElasticNet", ElasticNet()),
          ('KNN', KNeighborsRegressor()),
          ('CART', DecisionTreeRegressor()),
          ('RF', RandomForestRegressor()),
          #('SVR', SVR()),
          ('GBM', GradientBoostingRegressor()),
          ("XGBoost", XGBRegressor(objective='reg:squarederror')),
          ("LightGBM", LGBMRegressor())]
          # ("CatBoost", CatBoostRegressor(verbose=False))]

for name, regressor in models:
    rmse = np.mean(np.sqrt(-cross_val_score(regressor, X_train, y_train, cv=5, scoring="neg_mean_squared_error")))
    print(f"RMSE: {round(rmse, 4)} ({name}) ")

In [ ]:
df['SALARY_AVG_TL'].mean()

In [ ]:
df['SALARY_AVG_TL'].std()

In [ ]:
#DecisionTreeRegressor() MODELİNİN R-SQUARED DEGERİNE BAKIYORUM
model= DecisionTreeRegressor()
model.fit(X_train, y_train)
score = model.score(X_train, y_train)
print("R-squared:", score)


In [ ]:
#DECİSİON TREE REGRESSOR

In [ ]:
#BURADA MODELİ X_TRAİN'DE MODELİ EGİTTİM, X_TEST'TE TAHMİN ETTİM
model= DecisionTreeRegressor()
model.fit(X_train, y_train)
#y_pred=model.predict(df.drop(["SALARY_AVG_TL"], axis=1))
y_pred = model.predict(X_test)
y_pred

In [ ]:
#BU GRAFİKTE Y_TEST'TEKİ DEGERLER VE BUNLARIN TAHMİNLERİ VAR
x_ax = range(len(y_test))
plt.plot(x_ax, y_test, linewidth=1, label="original")
plt.plot(x_ax, y_pred, linewidth=1, label="predicted")
plt.title("Salary test and predicted data")
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.legend(loc='best',fancybox=True, shadow=True)
plt.grid(True)
plt.show()

In [ ]:
#Y_TEST VE Y_PRED DEGERLERİMİZİ AYRI BİR DATAFRAME'DE YANYANA GÖRELİM
veri=np.array(y_test)
veri

In [ ]:
y_pred

In [ ]:
dff=pd.DataFrame(data=veri, columns=["y_test"])
dff
dff["y_pred"]=  pd.DataFrame(y_pred,columns = ["y_pred"])
dff.head(40)

In [ ]:
#LOGİSTİCREGRESSİON MODELİNİ KURALIM

In [ ]:
log_model = LogisticRegression().fit(X, y)

log_model.intercept_  #sabit
log_model.coef_ #bagımsız degiskenlerin agırlıkları


y_pred = log_model.predict(X_test)#modeli kullanarak tahmin etme işlmei

y_pred[0:10] #tahmin ettiklerimiz


In [ ]:
#Y_TEST VE Y_PRED DEGERLERİMİZİ AYRI BİR DATAFRAME'DE YANYANA GÖRELİM
veri=np.array(y_test)
veri

In [ ]:
df2=pd.DataFrame(data=veri, columns=["y_test"])
df2
df2["y_pred"]=  pd.DataFrame(y_pred,columns = ["y_pred"])
df2.head(40)

In [ ]:
#random_user = X.sample(1, random_state=45)
#log_model.predict(random_user)

In [ ]:
#df.to_csv("step5.csv")